<h1> FLOW<h2>
<h3>> Certain amount of time is set to give use some time to get ready</h3>
<h3>> the trainer will then assume the pose that leads to the first part of the exercise</h3>
<h3>> an indicator will prompt the user to perform the exercise</h3>
<h3>> the user will have to stop in between exercise</h3>

    example:    
        push-up:    
            1) Starting Position:
            2) Push-Up Descent:
            3) Push-Up Ascent:  
            4) Stop momentarily, then it'll prompt to do it again   

<h3>> every start of the exercise will be recorded unto a txt file for the landmark coordinates</h3>


        

In [ ]:
# DATA COLLECTING
import cv2
import mediapipe as mp
from IPython.display import clear_output

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# cap = cv2.VideoCapture("vid4.mp4")
cap = cv2.VideoCapture(0)


# parameters:
# ================================================================================================================================
# timer to ready before posing/getting ready for the excercise
set_timer_start = 50
# ================================================================================================================================
# range of how much change will it be considered a movement
# lower -> small movement and jittery flaw of mediapipe will be detected
# higher -> it wont detect immediately important frames
# 0-1
change_range = .03
# ================================================================================================================================
# a counter for the amount of time for the wait time before performing the exercise again
between_exercise_ctr = 6



isInitiated = False
exercise_exectuted_ctr=0
prev_x = 0
prev_y = 0
no_movement_ctr = 0
movement_boolean = False
timer = 0

prev_array = []

for x in range(33):
    prev_array.append([0, 0])

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not ret:
        break

    timer += 1 
    landmark_all_present = False

    if timer == set_timer_start:
        file = open('base.txt', 'w')
        if isInitiated==False:
            file.write("START")
            file.write('\n')
        isInitiated = True

    if results.pose_landmarks:
        ctr_check=0

        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            if idx == 32:
                landmark_all_present = "Landmarks -> ALL PRESENT"
            else:
                landmark_all_present = "Landmarks -> SOME ARE MISSING"

            x = landmark.x
            y = landmark.y

#checks if no movement is detected-----------------------------------------------------------------------------------------------------------------------------------------
            if x >= (prev_array[idx][0] - change_range) and y <= (prev_array[idx][1] + change_range) and isInitiated:   
                print(idx,")",x,"---",prev_array[idx][0],'---> NO MOVEMENT')
                ctr_check+=1          
                
            else:
                print(idx,")",x,"---",prev_array[idx][0],'--->YES MOVEMENT',)
                movement_boolean = True
                no_movement_ctr = 0

            if ctr_check==32:
                no_movement_ctr += 1 

            if no_movement_ctr>=25:
                movement_boolean=False
            prev_array[idx] = [x, y]
        print('counter->',no_movement_ctr)
        print(movement_boolean)

#if movement is detected then it will record the coordinates into a text file-----------------------------------------------------------------------------------------------------
        if movement_boolean:
            text = "MOVEMENT"      
            if timer >= set_timer_start and file is not None:
                for landmark in prev_array:
                    # file.write("<")
                    file.write("|")
                    file.write(str(landmark[0]))
                    file.write("|")
                    file.write(str(landmark[1]))
                    # file.write(">")
                    # file.write("|")
                file.write('\n')

        if no_movement_ctr >= 5:    
            movement_boolean = False
            text = "NO MOVEMENT!"

        if no_movement_ctr == between_exercise_ctr:
            exercise_exectuted_ctr+=1
            file.write("END")  
            file.write('\n')
            file.write('\n')
            file.write("START")
            file.write('\n')



        if timer >= set_timer_start and no_movement_ctr >=between_exercise_ctr:
            cv2.putText(frame, "PERFORM THE EXERCISE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#Outputing important informations-----------------------------------------------------------------------------------------------------------------------------------------------
        cv2.putText(frame, str(movement_boolean), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"no_movement_ctr-> {no_movement_ctr}", (20, 37), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, str(landmark_all_present), (20, 54), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"timer-> {timer}", (20, 73), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"exercise exectuted-> {exercise_exectuted_ctr}", (20, 92), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        if timer < set_timer_start:
            cv2.putText(frame, "PREPARE YOUR STARTING POSE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
            

        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
        )

    cv2.imshow('MediaPipe Pose', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

if file is not None:
    file.close()
cap.release()
cv2.destroyAllWindows()


In [94]:
import numpy as np

def txt_pre_process(txt_file,label,simplify=False,simplify_level=4 ):
    label_array = []
    temp_feature_data = []
    temp_sequence_data = []
    batch_data = []

    with open(str(txt_file), 'r') as file:

        for line in file:
            values = line.strip().split('|')        

            temp_feature_data = []

            for value in values:            
                float_value = str(value)

                #FIRST PART OF THE SEQUENCE 
                if float_value == 'START':   
                    temp_sequence_data=[]
                                        
                elif float_value == 'END':              
                    batch_data.append(temp_sequence_data)
                    label_array.append(label)

            
                elif float_value != '' and float_value != 'START': 
                    if simplify:    
                        float_value = round(float(value),simplify_level)
                    else:
                        float_value = float(value)
                    temp_feature_data.append(float_value)

            if temp_feature_data!=[]:
                temp_sequence_data.append(temp_feature_data)

    label_array = np.array(label_array)
    return [batch_data,label_array]

#--------------------------------------------------------------------------- padding --------------------------------------------------------------------------------
# padding can be improved probably...by using sequence 
# minor issue:
# > is whether sequences had exceeded the intended number of sequences but is still right (it was performed right but slower(by an acceptable margin)) - not resolved
#    = temporary fix was just to truncate everything if it had exceeded the intended number of sequence for the sake of running it for now


def padding(pre_processed_input,optional_maxLength=0): 
    padded_sequences = []
    if optional_maxLength != 0:
        max_length = optional_maxLength
    else:
        max_length = max(len(sequence) for sequence in pre_processed_input)
    
    for sequence in pre_processed_input:        
        padding_length = max_length - len(sequence)
        if padding_length >= 0:
            padded_sequence = np.pad(sequence, ((0, padding_length), (0, 0)), mode='constant')
            
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    padded_sequences = np.array(padded_sequences)

    return padded_sequences

#--------------------------------------------------------------------------- padding --------------------------------------------------------------------------------



def concatenate_randomize_batches(base_input,base_label,concat_input,concat_label):
    combined_inputs = np.concatenate((base_input,concat_input), axis = 0)
    combined_label = np.concatenate((base_label,concat_label), axis = 0)
    indices = np.random.permutation(len(combined_inputs))
    randomized_inputs = combined_inputs[indices]
    randomized_label = combined_label[indices]
    return [randomized_inputs,randomized_label]





In [137]:
temp_correct = txt_pre_process('base(correct_execution).txt',1,True,4)
temp_wrong = txt_pre_process('base(wrong_execution).txt',0,True,4)

base_input = padding(temp_correct[0])
concat_input = padding(temp_wrong[0],31)

base_label = temp_correct[1]
concat_label = temp_wrong[1]

# optional if labels were not initially assigned
# correct_labels = np.ones(len(base_input))  # Assign label 1 to correct inputs
# wrong_labels = np.zeros(len(concat_input))  # Assign label 0 to wrong inputs

rand_batches=concatenate_randomize_batches(base_input,base_label,concat_input,concat_label)

print("-------------------checking:-------------------")
print("base_input->",base_input.shape)
print("concat_input->",concat_input.shape)
print("successfully merged and randomized(input) ->",rand_batches[0].shape)
print("successfully merged and randomized(label) ->",rand_batches[1].shape)

-------------------checking:-------------------
base_input-> (60, 31, 66)
concat_input-> (44, 31, 66)
successfully merged and randomized(input) -> (104, 31, 66)
successfully merged and randomized(label) -> (104,)


In [98]:
temp = txt_pre_process('base(correct_execution).txt',1)
inputs = padding(temp[0])
label = temp[1]

print("padded_sequences shape:", inputs.shape)
print("label shape:", label.shape)


padded_sequences shape: (60, 31, 66)
label shape: (60,)


In [99]:
np.array(base_input).shape

(60, 31, 66)

In [100]:
len(base_input)

60

In [123]:
base_input[0]

array([[0.4153, 0.0776, 0.4178, ..., 0.8023, 0.356 , 0.7986],
       [0.4192, 0.0777, 0.4205, ..., 0.8032, 0.3533, 0.7988],
       [0.4213, 0.0777, 0.422 , ..., 0.804 , 0.349 , 0.799 ],
       ...,
       [0.4228, 0.0771, 0.4256, ..., 0.8104, 0.356 , 0.8038],
       [0.4228, 0.0771, 0.4254, ..., 0.8098, 0.3559, 0.804 ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]])

In [106]:
test_label = [1] * len(base_input)
test_label = np.ones(len(test_label)) 

In [116]:
len(test_label)

60

In [164]:
X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.2, random_state=42)

In [ ]:
# version 1
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(31, 66)))
model.add(LSTM(128, return_sequences=True, activation='relu')) 
model.add(LSTM(64, return_sequences=False, activation='relu')) 
model.add(Dense(64, activation='relu')) 
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])

In [165]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


# version 2
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', dropout=0.2, recurrent_dropout=0.2, input_shape=(31, 66)))
model.add(LSTM(128, return_sequences=True, activation='relu')) 
model.add(LSTM(64, return_sequences=False, activation='relu')) 
model.add(Dense(64, activation='relu')) 
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])



In [142]:
print(rand_batches[0].shape)
print(rand_batches[1].shape)

(104, 31, 66)
(104,)


In [141]:
print(base_input.shape)
print(test_label.shape)

(60, 31, 66)
(60,)


In [ ]:
model.fit(base_input,test_label,epochs=10)

In [175]:
# model.fit(rand_batches[0],rand_batches[1],epochs=100)
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
3/3 [==============================] - 3s 944ms/step - loss: 1.1314 - accuracy: 0.5301 - binary_accuracy: 0.5301 - val_loss: 0.9208 - val_accuracy: 0.3333 - val_binary_accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 3s 885ms/step - loss: 0.8739 - accuracy: 0.4458 - binary_accuracy: 0.4458 - val_loss: 0.8130 - val_accuracy: 0.3333 - val_binary_accuracy: 0.3333
Epoch 3/100
3/3 [==============================] - 2s 766ms/step - loss: 0.7770 - accuracy: 0.4458 - binary_accuracy: 0.4458 - val_loss: 0.7723 - val_accuracy: 0.3333 - val_binary_accuracy: 0.3333
Epoch 4/100
3/3 [==============================] - 2s 875ms/step - loss: 0.7395 - accuracy: 0.4458 - binary_accuracy: 0.4458 - val_loss: 0.7418 - val_accuracy: 0.3333 - val_binary_accuracy: 0.3333
Epoch 5/100
3/3 [==============================] - 2s 817ms/step - loss: 0.7147 - accuracy: 0.4458 - binary_accuracy: 0.4458 - val_loss: 0.7183 - val_accuracy: 0.3333 - val_binary_accuracy: 0.3333
Epoch 6/100
3/3

In [177]:
pred = model.predict(concat_input)
temp = np.sum(pred) / len(pred)
print(temp)



2/2 [==============================] - 0s 96ms/step
0.024964636022394352


: 

In [110]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

initial_input = rand_batches

max_length = len(initial_input[0][0])
# input_test = inputs
# label_test = label



model = Sequential()
model.add(keras.Input(shape=(max_length, 66))) # seq_length, input_size

model.add(Bidirectional(LSTM(66, return_sequences=True, activation='relu'))) # N, 128
model.add(Bidirectional(LSTM(66, return_sequences=True, activation='relu')) )
model.add(Bidirectional(LSTM(66, return_sequences=True, activation='relu')) )
model.add(Bidirectional(LSTM(66, return_sequences=True, activation='relu')) )
model.add(Bidirectional(LSTM(66, activation='relu'))) # N, 128
model.add(Dense(132,activation='softmax'))
model.add(Dense(1,activation='sigmoid'))



print(model.summary())

loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.005)


X_train, X_test, y_train, y_test = train_test_split(initial_input[0], initial_input[1], test_size=0.15, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 31, 132)          70224     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 31, 132)          105072    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 31, 132)          105072    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 31, 132)          105072    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 132)              1

d:\Anaconda\envs\mediapipe\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [51]:


model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=100, validation_data=(X_test, y_test))

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=10, batch_size=5, validation_data=(X_test, y_test))


Epoch 1/100
1/1 [==============================] - 20s 20s/step - loss: 0.6931 - accuracy: 0.5909 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 0.6920 - accuracy: 0.5909 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 0.6908 - accuracy: 0.5909 - val_loss: 0.6923 - val_accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 0.6873 - accuracy: 0.5909 - val_loss: 0.6846 - val_accuracy: 0.5625
Epoch 5/100
1/1 [==============================] - 2s 2s/step - loss: 0.6816 - accuracy: 0.5795 - val_loss: 0.6856 - val_accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 2s 2s/step - loss: 0.6773 - accuracy: 0.5795 - val_loss: 0.7102 - val_accuracy: 0.4375
Epoch 7/100
1/1 [==============================] - 2s 2s/step - loss: 0.6927 - accuracy: 0.5455 - val_loss: 0.6989 - val_accuracy: 0.5000
Epoch 8/100
1/1 [===============

In [52]:
wrong = txt_pre_process('base(wrong_execution).txt',0)
wrong_inputs = padding(wrong[0])
wrong_label = wrong[1]


correct = txt_pre_process('base(correct_execution).txt',1)
correct_inputs = padding(wrong[0])
correct_label = wrong[1]

In [122]:
ctr_0=0
ctr_1=0
inputss = wrong
for x in range(len(inputss[0])):
    input_sequence = inputss[0][x]
    length_input = max_length - len(input_sequence)
    if length_input>=0:
        padded_input = np.pad(input_sequence, ((0, max_length - len(input_sequence)), (0, 0)), mode='constant')
    else:
        padded_input = input_sequence[:max_length]

    reshaped_input = np.reshape(padded_input, (1, max_length, 66))


    # print(reshaped_input.shape )

    logits = model.predict(reshaped_input )

    threshold = 0.8
    binary_predictions = (logits > threshold).astype(int)

    print("Predicted logits:", logits)
    print("Binary predictions:", binary_predictions[0][0])
    if binary_predictions[0][0] == 1:
        ctr_1 = ctr_1 + 1
    else:
        ctr_0 = ctr_0 + 1

print('ctr_1 ->',ctr_1)
print('ctr_0 ->',ctr_0)

NameError: name 'wrong' is not defined

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [8]:
def draw_landmarks(image,results):
    # mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [6]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [11]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness = 1, circle_radius = 1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness = 1, circle_radius = 1))
    
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness = 1, circle_radius = 4),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness = 1, circle_radius = 2))

    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness = 1, circle_radius = 4),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness = 1, circle_radius = 2))
    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness = 1, circle_radius = 4),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness = 1, circle_radius = 2))

In [84]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame,holistic)

        draw_styled_landmarks(image,results)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [37]:
print(results.right_hand_landmarks)


None


In [90]:
temp_compile_landmarks = []
temp_compile_landmarks_lh = []
temp_compile_landmarks_rh = []

# for coor in results.pose_landmarks.landmark:
#      temp = np.array([coor.x,coor.y,coor.z,coor.visibility])
#      temp_compile_landmarks.append(temp)
def data_extractions(results):
    temp_compile_landmarks = np.array([[coor.x,coor.y,coor.z,coor.visibility] for res in results.pose_landmarks.landmark] if results.left_hand_landmarks else np.zeros(132) ).flatten() 
    # temp_compile_landmarks_lh = np.array([[coor.x,coor.y,coor.z,coor.visibility] for res in results.left_hand_landmarks.landmark] if results.left_hand_landmarks else np.zeros(21*3)).flatten()
    # temp_compile_landmarks_rh = np.array([[coor.x,coor.y,coor.z,coor.visibility] for res in results.right_hand_landmarks.landmark] if results.right_hand_landmarks else np.zeros(21*3)).flatten()
    return temp_compile_landmarks

